# Recommender system for a medical supplies company

<img src="https://media.licdn.com/dms/image/D5612AQEPazd7U9Kf1Q/article-cover_image-shrink_720_1280/0/1681454275602?e=2147483647&v=beta&t=e9q_u8PmwiXaY1uWZap8rM8RaZuIQOOFf4G97AJxmQE"  style="width: 100%; height: auto;">

## Proje Amacı
Projenin hedefi, şirketin ek satışlar elde etmesine yardımcı olmak için bir öneri sistemi geliştirmektir.

* Veri Hazırlama ve Modelleme Aşamaları
  
* Eksik Veriler: Verideki eksik kısımların belirlenmesi ve bu eksikliklerin giderilmesi.
  
* En Popüler Ürünü Bulma, Popular Based, Müşterilere tavsiye edilecek populer ürünlerin bulunması.

* En Fazla Satın Alma Yapan Şirket:En çok satın alma yapan müşteri veya şirketin belirlenmesi.

* Matris Faktörizasyonu Kullanarak Öneri Sistemi:Kullanıcı ve ürün siparişine dayanarak matris faktörizasyonu ile bir öneri tavsiye geliştirilmesi.

* Ürün Açıklamaları için Kosinüs Benzerliği:Ürün açıklamaları arasındaki benzerlikleri ölçmek için kosinüs benzerliği yönteminin kullanılması.

* Bu aşamalar tamamlandığında, şirketin satışlarını artırmaya yönelik etkili bir öneri sistemi elde edilmiş olacak.

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)  

In [3]:
df = pd.read_csv('PBL 5 recommendation data.csv', encoding='iso-8859-1')
pd.set_option('display.max_columns',200)

C:\Users\İdeaPad\AppData\Local\Temp\ipykernel_17808\717764142.py:1: DtypeWarning: Columns (20,33,73,106,158) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('PBL 5 recommendation data.csv', encoding='iso-8859-1')


In [6]:
df.head()

,Customers.id,Customers.fname,Customers.lname,Customers.company,Customers.create_date,Customers.status,Customers.mailing,Customers.reminders,Customers.tax_exempt,Customers.account_id,Customers.sales_rep,Customers.rewards,Customers.profile_id,Customers.last_modified,Customers.customer_type,Orders.id,Orders.customer_id,Orders.fname,Orders.lname,Orders.company,Orders.order_number,Orders.reorder_id,Orders.external_source,Orders.external_id,Orders.currency,Orders.sales_rep,Orders.subtotal,Orders.tax,Orders.shipping,Orders.coupon_id,Orders.coupon_amount,Orders.gift_id,Orders.gift_amount,Orders.fee_name,Orders.fee_amount,Orders.discount_name,Orders.discount_amount,Orders.total,Orders.balance_due,Orders.shipping_carrier,Orders.shipping_method,Orders.shipping_trans,Orders.shipping_flags,Orders.weight,Orders.tracking,Orders.payment_status,Orders.payment_date,Orders.payment_user,Orders.payment_type,Orders.payment_method,Orders.payment_amount,Orders.purchase_order,Orders.payment_id,Orders.payment_code,Orders.payment_ref,Orders.status,Orders.placed_date,Orders.updated_date,Orders.shipped_date,Orders.comments,Orders.notes,Orders.registry_id,Orders.gift_message,Orders.website,Orders.mailing,Orders.flags,Orders.partial_ship,Orders.customer_type,Order_Items.id,Order_Items.parent,Order_Items.product_id,Order_Items.product_name,Order_Items.attributes,Order_Items.attribute_names,Order_Items.attribute_prices,Order_Items.qty,Order_Items.price,Order_Items.cost,Order_Items.registry_item,Order_Items.related_id,Order_Items.reorder_frequency,Order_Items.account_id,Order_Items.flags,Products.id,Products.status,Products.product_type,Products.template,Products.vendor,Products.import_id,Products.name,Products.display_name,Products.menu_name,Products.list_price,Products.price,Products.sale_price,Products.cost,Products.flags,Products.left_flag,Products.right_flag,Products.last_modified,Products.taxable,Products.shopping_gtin,Products.shopping_brand,Products.shopping_mpn,Products.shopping_gender,Products.shopping_color,Products.shopping_age,Products.shopping_flags,Products.amazon_asin,Products.amazon_type,Products.amazon_item_type,Products.amazon_price,Products.google_shopping_id,Products.google_shopping_type,Products.google_shopping_cat,Products.google_adwords,Products.shopping_cat,Products.shopping_type,Products.pricegrabber_cat,Products.shopzilla_cat,Products.thefind_cat,Products.quickbooks_id,Products.qb_edit_sequence,Products.price_break_type,Products.price_breaks,Products.short_description,Products.long_description,Products.websites,Products.video,Products.audio,Products.seo_title,Products.seo_description,Products.seo_keywords,Products.seo_header,Products.seo_footer,Products.seo_url,Products.seo_category,Products.unit,Products.packaging,Products.display_packaging,Products.multiple,Products.length,Products.width,Products.height,Products.rx,Products.latex,Products.upc,Products.msds_link,Products.msds_label,Products.lit_link,Products.lit_label,Products.hcpcs,Products.case_qty,Products.markup,Products.override_markup,Products.notes,Products.import_flags,Products.map_price,Products.features_title,Products.warranty,Products.hygienic,Products.default_quantity,Products.user_size,Products.assembly,Products.installation,Products.shipping_length,Products.shipping_width,Products.shipping_height,Products.shipping_weight,Products.handling_time,Products.rotation_link,Products.google_shopping_label,Products.product_option,Products.size,Products.material,Products.arm_style,Products.leg_style,Products.seat_size,Products.family_id,Products.saved_status,Products.freight_cost
0,797,Christy,Dill,Company0,1426018724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1437764306,0.0,3758,797,Christy,Dill,Company0,3758,NaN,NaN,NaN,USD,NaN,57.20,0.0,9.95,13.0,2.86,NaN,NaN,NaN,NaN,NaN,NaN,64.29,NaN,fedex,11|Ground,NaN,NaN,NaN,5.7204E+14,3.0,1.426019e+09,NaN,authorize.net,NaN,64.29,NaN,6993607863,510142,NaN,1,1426019099,1.438868e+09,1.426101e+09,NaN,Insured By Eye4Fraud,NaN,NaN,NaN,NaN,NaN,NaN,0.0,528

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4194 entries, 0 to 4193
Columns: 181 entries, Customers.id to Products.freight_cost
dtypes: float64(98), int64(10), object(73)
memory usage: 5.8+ MB


In [8]:
df.shape

(4194, 181)

In [9]:
df.isnull().sum()

Customers.id                         0
Customers.fname                      0
Customers.lname                      0
Customers.company                 3467
Customers.create_date                0
Customers.status                  4093
Customers.mailing                 2080
Customers.reminders               4194
Customers.tax_exempt              4189
Customers.account_id              4191
Customers.sales_rep               4194
Customers.rewards                 4194
Customers.profile_id              4194
Customers.last_modified              0
Customers.customer_type            423
Orders.id                            0
Orders.customer_id                   0
Orders.fname                         0
Orders.lname                         0
Orders.company                    3466
Orders.order_number                  0
Orders.reorder_id                 4112
Orders.external_source            3829
Orders.external_id                4189
Orders.currency                      0
Orders.sales_rep         

* Çok fazla boş sutun var ve bazı sutunların sadece %20 veya ,%30 u dolu,  bu sutunları silmek istedim

In [10]:
# Eksik değer yüzdelerini hesaplama
missing_percentage = df.isnull().mean() * 100

# %30'dan fazla eksik değere sahip sütunları silme
columns_to_drop = missing_percentage[missing_percentage > 30].index
df = df.drop(columns=columns_to_drop)

In [12]:
df.columns

Index(['Customers.id', 'Customers.fname', 'Customers.lname',
       'Customers.create_date', 'Customers.last_modified',
       'Customers.customer_type', 'Orders.id', 'Orders.customer_id',
       'Orders.fname', 'Orders.lname', 'Orders.order_number',
       'Orders.currency', 'Orders.subtotal', 'Orders.tax', 'Orders.shipping',
       'Orders.total', 'Orders.shipping_carrier', 'Orders.shipping_method',
       'Orders.tracking', 'Orders.payment_status', 'Orders.payment_date',
       'Orders.payment_type', 'Orders.payment_amount', 'Orders.payment_id',
       'Orders.payment_code', 'Orders.status', 'Orders.placed_date',
       'Orders.updated_date', 'Orders.shipped_date', 'Orders.customer_type',
       'Order_Items.id', 'Order_Items.parent', 'Order_Items.product_id',
       'Order_Items.product_name', 'Order_Items.qty', 'Order_Items.price',
       'Order_Items.cost', 'Products.id', 'Products.status',
       'Products.template', 'Products.vendor', 'Products.import_id',
       'Products.name

In [13]:
df.shape

(4194, 72)

In [15]:
df["Products.name"].value_counts()

Products.name
Emesis Bags,Blue,36.000 OZ                                                                                            103
MoliCare Disposable Super Plus Briefs,27"-47"                                                                          71
Aluminum Transport Chair with 12" Wheels,Red,F: 8   R: 12                                                              67
Aluminum Transport Chair with 12" Wheels,Blue,F: 8   R: 12                                                             67
K1 Basic Extra-Wide Wheelchairs                                                                                        54
Silent Knight Pill Crushers                                                                                            53
Silent Knight Pill Crusher Pouches                                                                                     51
K1 Basic Wheelchairs                                                                                                   40
MoliCare D

In [17]:
df["Orders.currency"].value_counts()
#herkes dolar ile ödeme yapmış

Orders.currency
USD    4194
Name: count, dtype: int64

## Popularity Base

-Önce popularity base e bakmamız lazım yanı en popüler en çok satılan ürün ne? (100den fazla olması şart)

-Hem size hem mean yüksek olan ürün "Medline MoliCare Super Plus Disposable Adult Diapers" çıktı,

-size 500 olan ürün vardı ama sadece 1 şirket satın almış, ortalamasıda yüksek olmalı popularite için

In [27]:
# df deki siparişleri, ürün adlarına göre gruplamyıp her ürün için sipariş sayısı,ortalama ve ortalama sipariş sayısı
grouped1=df.groupby('Order_Items.product_name').agg({'Order_Items.qty':[np.size,np.sum,np.mean]})

C:\Users\İdeaPad\AppData\Local\Temp\ipykernel_17808\420128192.py:2: FutureWarning: The provided callable <function sum at 0x000001313BA86020> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  grouped1=df.groupby('Order_Items.product_name').agg({'Order_Items.qty':[np.size,np.sum,np.mean]})
C:\Users\İdeaPad\AppData\Local\Temp\ipykernel_17808\420128192.py:2: FutureWarning: The provided callable <function mean at 0x000001313BA86E80> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  grouped1=df.groupby('Order_Items.product_name').agg({'Order_Items.qty':[np.size,np.sum,np.mean]})


In [28]:
grouped1

Order_Items.qty       \
                                                              size  sum   
Order_Items.product_name                                                  
 BUCKET, REPLACEMENT FOR MDS89668XW                              1    1   
1 Box Of Medline Commode Liners with Absorbent Pad               1    2   
1 Pad of CURAD Mediplast Wart Pad                                1    1   
1-Ply Blockade AngelStat Surgical Gowns, Ceil B...               1    1   
1/2" self-adhesive hook material, 25 yard dispe...               1    1   
1/2" self-adhesive loop material, 25 yard dispe...               1    1   
100% Cotton Unisex Reversible Hyperbaric Scrub ...               2    3   
100% Cotton Unisex Reversible Tops, Ceil Blue, MHS               1    3   
10215-s                                                          1    1   
12112-3                                                          1    1   
12124-3                                                          1    1   
18" Hamper Stands, Standard                                      6   11   
19" Flyweight Lightweight Blue Transport Wheelc...               1    1   
2" self-adhesive loop material, 10 yard dispens...               1    1   
2" self-adhesive loop material, 25 yard dispens...               1    1   
2-Ply Tissue/Poly Professional Towels, White, N...               2  220   
2000 Extra-Wide Wheelchairs                                      3    3   
2000 Wheelchairs                                                 1    1   
24 Hour Urine Collection Bottle, 3000.0 ML                       2    5   
3 Pack, Hospital Receiving Blankets, Baby Blank...               1    1   
3 Pack, Hospital Receiving Blankets, Baby Blank...               1    1   
3 Wheel Flame Blue Rollator Walker with Basket ...               2    2   
3 Wheel Flame Red Rollator Walker with Basket T...               1    1   
3 Wheel Walker Rollator with Basket Tray and Po...               1    1   
3 Wheel Walker Rollator with Basket Tray and Po...               2    2   
3" Wheels For Walkers                                            2    2   
3/4" Guardian Utility Tips, Gray                                 1    1   
4-Panel Abdominal Binders, 2X-Large                              4    8   
4-Panel Abdominal Binders, Large/X-Large                         4   12   
5" Wheels For Walkers                                            2    2   
6 Cylinder Oxygen Carrier, M9 Cylinders                          1    1   
6' CPAP Tube                                                     1    1   
6-spring hand grip (each)                                        2    2   
6291hda Universal wheels part                                    1    1   
8" Rear Walker Wheel Attachment                                  3    3   
A&D Ointment by H & H Laboratories Inc, 384.00 ML                1    1   
Accu-Therm Hot/Cold Gel Packs                                    2    4   
Accu-Therm Insulated Hot Pack                                    1    2   
Accu-Therm Non-Insulated Hot Pack                                1    1   
Accutouch Chemo Nitrile Exam Gloves, Blue, Medium                1    1   
Active Liquid Protein Nutritional Supplement, 0.0                1    1   
Active Liquid Protein Nutritional Supplement, 3...               1    1   
AcuForce&reg; 7.0 Massage Tool                                   1    1   
Adaptive Eating Utensils - Adaptive Fork                         1    1   
Addipak Sterile Saline Solution                                  1    1   
Adjustable Angle Rotating Suction Cup Grab Bar                   1    1   
Adjustable Height ArmRest AK5ADJARMKIT                           1    1   
Adjustable Height Bathtub Grab Bar Safety Rail                   1    2   
Adjustable Height Home Bed Assist Handle                         6    6   
Adjustable Height Rollator with 6" Wheels, Red                   1    1   
Adjustable Height Teak Bath Bench Stool, Rectan...               3    3   
A

In [32]:
sorted_grouped.head()
# ortalaması en yüksek olanı bulunca en popüleri bulmuş oluruz, en yüksek TruTemp çıktı ama 1 şirket almış sadece,size de yüksek olanı bulmalıyız

Order_Items.qty            
                                                              size  sum   mean
Order_Items.product_name                                                      
TruTemp                                                          1  500  500.0
2-Ply Tissue/Poly Professional Towels, White, N...               2  220  110.0
Sanitary Pads with Adhesive & Wings                              2  133   66.5
Remedy Basics Moisturizing Body Lotion, 118.00 ML                1   40   40.0
MPHSAFETY21Z                                                     1   24   24.0

* Hem ortalaması hem size yüksek olan bu ürün "yetişkin bağımsızlık ürünü"

In [37]:
grouped1 = df.groupby('Order_Items.product_name').agg({'Order_Items.qty': [np.size, np.sum, np.mean]})
sorted_grouped = grouped1.sort_values(by=[('Order_Items.qty', 'size'), ('Order_Items.qty', 'mean')], ascending=False)

C:\Users\İdeaPad\AppData\Local\Temp\ipykernel_17808\1515567149.py:1: FutureWarning: The provided callable <function sum at 0x000001313BA86020> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  grouped1 = df.groupby('Order_Items.product_name').agg({'Order_Items.qty': [np.size, np.sum, np.mean]})
C:\Users\İdeaPad\AppData\Local\Temp\ipykernel_17808\1515567149.py:1: FutureWarning: The provided callable <function mean at 0x000001313BA86E80> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  grouped1 = df.groupby('Order_Items.product_name').agg({'Order_Items.qty': [np.size, np.sum, np.mean]})


In [38]:
sorted_grouped.head()

Order_Items.qty       \
                                                              size  sum   
Order_Items.product_name                                                  
Medline MoliCare Super Plus Disposable Adult Di...              53  164   
Silent Knight Pill Crusher Pouches                              48   72   
Medline Emesis/Barf Bags, throw up bags, Blue -...              47   58   
Medline Deluxe Aluminum Transport Chair With Ha...              46   47   
K1 Basic Extra-Wide Wheelchairs                                 36   39   

                                                              
                                                        mean  
Order_Items.product_name                                      
Medline MoliCare Super Plus Disposable Adult Di...  3.094340  
Silent Knight Pill Crusher Pouches                  1.500000  
Medline Emesis/Barf Bags, throw up bags, Blue -...  1.234043  
Medline Deluxe Aluminum Transport Chair With Ha...  1.021739  
K1 Basic Extra-Wide Wheelchairs                     1.083333

* Spopularity base göre Yetişkin bezi alan müşteriye sırasıyla bu ürnleri tavsiye edebiliriz.

* Sonuç olarak sadece popularity base uygulayabildim.
* maxrix factorization uygulayamadım Pivot table kodum çalışmadı.
* Cosinus Distance uygulamaya çalışırken yine bir problemle karşılaştım.